In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

import pandas as pd
import glob
import matplotlib.pyplot as plt

## Google Colab

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# 先行研究

## 発話エンコーダ設定

In [ ]:
utter_embedding = tf.keras.layers.Embedding(1000, 300)
utter_gru = tf.keras.layers.GRU(512)

## 文脈エンコーダ設定

In [ ]:
context_gru = tf.keras.layers.GRU(513)

## 対話行為エンコーダ設定

In [ ]:
dialog_embedding = tf.keras.layers.Embedding(1000, 100)
dialog_gru = tf.keras.layers.GRU(128)

## 分類器設定

## 損失関数設定

## 最適化関数設定

## 実行(前処理)

## filepath(colab)

In [55]:
train_data = "drive/My Drive/研究/swda_data/train_set.txt"
#test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
val_data = "drive/My Drive/研究/swda_data/val_set.txt"

## filepath(local)

In [4]:
train_data = "../Switchboard-Corpus/swda_data/train_set.txt"
#test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
val_data = "../Switchboard-Corpus/swda_data/val_set.txt"

In [5]:
train_dataset = []
f = open(train_data, "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [9]:
val_dataset = []
f = open(val_data, "r", encoding='utf-8')
for row in f:
    val_dataset.append(row.strip())
f.close()

## 発話者,発話,発話ラベル(訓練データ)

In [6]:
utter_user = []
utter = []
utter_label = []

for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


## 発話者,発話,発話ラベル(検証データ)

In [10]:
val_utter_user = []
val_utter = []
val_utter_label = []

for j in val_dataset:
    for i,v in enumerate(j.split("|")):
        if i == 0:
            val_utter_user.append(v)
        elif i==1:
            val_utter.append(v)
        else:
            val_utter_label.append(v)
print("finish")

finish


## データセット化

In [11]:
utter_train_dataset = tf.data.Dataset.from_tensor_slices((utter, utter_label))
utter_val_dataset = tf.data.Dataset.from_tensor_slices((val_utter, val_utter_label))

In [12]:
text, label = next(iter(utter_train_dataset))
print(text)
print(label)

tf.Tensor(b'Okay.', shape=(), dtype=string)
tf.Tensor(b'fo_o_fw_"_by_bc', shape=(), dtype=string)


## 分かち書き

In [88]:
## ボキャブラリーリスト
vocabulary_set = set()
## トークナイザー
tokenizer = tfds.features.text.Tokenizer()

## 分かち書き
for text_tensor,_ in utter_train_dataset:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)
## ボキャブラリーリスト作成
vocab_size = len(vocabulary_set)

## エンコード(トークンに変換する)

In [90]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)

In [91]:
def encode(token, label):
    token = encoder.encode(token.numpy())
    label = np.array([label])
    return token, label

@tf.function
def tf_encoder(utter, label):
    return tf.py_function(encode, [utter, label],[tf.int64, tf.int64])

MAX_LENGTH = 40
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [98]:
train_data_set = utter_train_dataset.map(tf_encoder)
train_data_set = train_data_set.filter(filter_max_length)
train_data_set = train_data_set.cache()
train_data_set = train_data_set.padded_batch(BATCH_SIZE, padded_shapes=(100,100))
train_data_set.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: ((None, 100), (None, 100)), types: (tf.int64, tf.int64)>

In [100]:
for (batch, (inp, tar)) in enumerate(train_data_set):
  print(inp)

InvalidArgumentError: ignored

## パラメータ設定

In [39]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [42]:
## 訓練データセット作成

In [14]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

### ウィンドウサイズ

In [ ]:
window_size = 5

### 実行モデル

In [49]:
model = tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size, 300),
                            tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512)),
                            tf.keras.layers.Dense(512, activation='relu'),
                            tf.keras.layers.Dense(1)
])

In [53]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [54]:
history = model.fit(train_dataset, epochs=10)

Epoch 1/10


ValueError: ignored